https://docs.amd.com/bundle/ROCm-Deep-Learning-Guide-v5.3/page/Frameworks_Installation.html

In [12]:
%cd /mnt/data_drive/CSU_PhD/research/software/dataset/SROIE_PICK

/mnt/data_drive/CSU_PhD/research/software/dataset/SROIE_PICK


In [2]:
## Creating folders for preprocessed dataset
!rm -rf boxes_and_transcripts images entities 
!mkdir boxes_and_transcripts images entities

In [ ]:
#!sudo apt install python3.8-dev
#!/usr/bin/python3.8 -m pip install packaging
#!/usr/bin/python3.8 -m pip install ipykernel -U --user --force-reinstall
!/usr/bin/python3.8 -m pip install pandas

In [1]:
## Script for preprocessing dataset
import os
import pandas
import json
import csv
import shutil

In [2]:
## Input dataset
data_path = "/mnt/data_drive/CSU_PhD/research/software/dataset/ICDAR-2019-SROIE/data/"
box_path = data_path + "box/"
img_path = data_path + "img/"
key_path = data_path + "key/"

## Output dataset
out_boxes_and_transcripts = "./boxes_and_transcripts/"
out_images = "./images/"
out_entities  = "./entities/"

In [5]:
train_samples_list =  []
for file in os.listdir(data_path + "box/"):
  
  ## Reading csv
  with open(box_path +file, "r") as fp:
    reader = csv.reader(fp, delimiter=",")
    ## arranging dataframe index ,coordinates x1_1,y1_1,x2_1,y2_1,x3_1,y3_1,x4_1,y4_1, transcript
    rows = [[1] + x[:8] + [','.join(x[8:]).strip(',')] for x in reader] 
    df = pandas.DataFrame(rows)
  
  ## including ner label dataframe index ,coordinates x1_1,y1_1,x2_1,y2_1,x3_1,y3_1,x4_1,y4_1, transcript , ner tag
  df[10] = 'other'  
  
  ##saving file into new dataset folder
  jpg = file.replace(".csv",".jpg")
  entities = json.load(open(key_path+file.replace(".csv",".json")))
  for key,value in sorted(entities.items()):
    idx = df[df[9].str.contains('|'.join(map(str.strip, value.split(','))))].index
    df.loc[idx, 10] = key

  shutil.copy(img_path +jpg, out_images)
  with open(out_entities + file.replace(".csv",".txt"),"w") as j:  
    print(json.dumps(entities), file=j)
  
  df.to_csv(out_boxes_and_transcripts+file.replace(".csv",".tsv"),index=False,header=False, quotechar='',escapechar='\\',quoting=csv.QUOTE_NONE, )
  train_samples_list.append(['receipt',file.replace('.csv','')])
train_samples_list = pandas.DataFrame(train_samples_list)
train_samples_list.to_csv("train_samples_list.csv")

/usr/local/lib/python3.8/dist-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [ ]:
## document_type, file_name
train_samples_list.head()

In [7]:
#%cd /mnt/data_drive/CSU_PhD/research/software/PICK-pytorch/data/data_examples_root

/mnt/data_drive/CSU_PhD/research/software/PICK-pytorch/data/data_examples_root


In [14]:
from sklearn.model_selection import train_test_split
train_test = pandas.read_csv("train_samples_list.csv",dtype=str)
train, test= train_test_split(train_test,test_size=0.2,random_state = 42)

**Copy train data into PICK-pytorch data folder**

In [9]:
for index, row in train.iterrows():
  shutil.copy(out_boxes_and_transcripts+str(row[2])+".tsv",
              '/mnt/data_drive/CSU_PhD/research/software/PICK-pytorch/data/data_examples_root/boxes_and_transcripts/')
  shutil.copy(out_images+str(row[2])+".jpg",
              '/mnt/data_drive/CSU_PhD/research/software/PICK-pytorch/data/data_examples_root/images/')
  shutil.copy(out_entities +str(row[2])+".txt", 
              '/mnt/data_drive/CSU_PhD/research/software/PICK-pytorch/data/data_examples_root/entities/')

train.drop(['Unnamed: 0'], 
           axis = 1,
           inplace = True)
train.reset_index(inplace= True)
train.drop(['index'], 
           axis = 1,
           inplace = True)
train.to_csv("/mnt/data_drive/CSU_PhD/research/software/PICK-pytorch/data/data_examples_root/train_samples_list.csv",
             header = False)

**Copy test data into PICK-pytorch data folder**

In [ ]:
!rm -rf /mnt/data_drive/CSU_PhD/research/software/PICK-pytorch/data/test_data_example/entities/
!rm -rf /mnt/data_drive/CSU_PhD/research/software/PICK-pytorch/data/test_data_example/images/
!rm -rf /mnt/data_drive/CSU_PhD/research/software/PICK-pytorch/data/test_data_example/boxes_and_transcripts/
!mkdir -p /mnt/data_drive/CSU_PhD/research/software/PICK-pytorch/data/test_data_example/entities/
!mkdir -p /mnt/data_drive/CSU_PhD/research/software/PICK-pytorch/data/test_data_example/images/
!mkdir -p /mnt/data_drive/CSU_PhD/research/software/PICK-pytorch/data/test_data_example/boxes_and_transcripts/

In [15]:
for index, row in test.iterrows():
  shutil.copy(out_boxes_and_transcripts+str(row[2])+".tsv",'/mnt/data_drive/CSU_PhD/research/software/PICK-pytorch/data/test_data_example/boxes_and_transcripts/')
  shutil.copy(out_images+str(row[2])+".jpg",'/mnt/data_drive/CSU_PhD/research/software/PICK-pytorch/data/test_data_example/images/')
  shutil.copy(out_entities +str(row[2])+".txt", '/mnt/data_drive/CSU_PhD/research/software/PICK-pytorch/data/test_data_example/entities/')

test.drop(['Unnamed: 0'], axis = 1,inplace = True)
test.reset_index(inplace= True)
test.drop(['index'], axis = 1,inplace = True)
test.to_csv("/mnt/data_drive/CSU_PhD/research/software/PICK-pytorch/data/test_data_example/test_samples_list.csv",header = False)

In [ ]:
## Removing data once it is copied into PICK-pytorch data folder
!rm -rf ./boxes_and_transcripts
!rm -rf ./images
!rm -rf ./entities

In [1]:
%cd /mnt/data_drive/CSU_PhD/research/software/PICK-pytorch

/mnt/data_drive/CSU_PhD/research/software/PICK-pytorch


In [18]:
%%writefile utils/entities_list.py
# -*- coding: utf-8 -*-
# @Author: Wenwen Yu
# @Created Time: 7/8/2020 9:34 PM

Entities_list = [
    "company",
    "address",
    "date",
    "total"
]

Overwriting utils/entities_list.py


In [13]:
# export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/opt/rocm/lib:/usr/lib/x86_64-linux-gnu
# sudo ldconfig
# ImportError: libMIOpen.so.1: cannot open shared object file: No such file or directory
# !sudo apt install miopen-hip miopen-hip-dev

# !sudo ln -s /opt/rocm-5.3.0/lib/libroctx64.so.4.1.0 /opt/rocm-5.3.0/lib/libroctx64.so.1
# !sudo ln -s /opt/rocm-5.3.0/lib/libroctracer64.so.4.1.0 /opt/rocm-5.3.0/lib/libroctracer64.so.1


# ImportError: librccl.so.1: cannot open shared object file: No such file or directory
# !wget http://mirrors.edge.kernel.org/ubuntu/pool/main/g/gcc-10/gcc-10-base_10-20200411-0ubuntu1_amd64.deb
# !wget http://mirrors.xmission.com/ubuntu/pool/main/g/gcc-10/libgcc-s1_10-20200411-0ubuntu1_amd64.deb

# !sudo dpkg -i ~/Downloads/gcc-10-base_10-20200411-0ubuntu1_amd64.deb
# !sudo dpkg -i ~/Downloads/libgcc-s1_10-20200411-0ubuntu1_amd64.deb
# !sudo apt install rccl

# ImportError: libtinfo.so.6: cannot open shared object file: No such file or directory
# !sudo apt install libtinfo-dev #Check with apt-cache policy libtinfo-dev first to make sure
# !sudo ln -s /usr/lib/x86_64-linux-gnu/libtinfo.so /usr/lib/x86_64-linux-gnu/libtinfo.so.5

# ImportError: /usr/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.26' not found
# !strings /usr/lib/x86_64-linux-gnu/libstdc++.so.6 | grep -i glib
# !sudo add-apt-repository ppa:ubuntu-toolchain-r/test
# !sudo apt update
# !sudo apt upgrade libstdc++6

#ImportError: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.29' not found 
#



In the docker server, since the initialization of the VS code is via the add-on it doesn't restart the VS Code.
Current environment is the virtual one,

```
$ sudo apt install jupyter # Install jupyter
$ jupyter notebook --allow-root # Allow to start jupyter server as root
```

Then click on the kernel selection for the ipython notebook. It will allow you to select the new notebook server.
Select that server and start the process.

In [5]:
!python -m pip install -r requirements.txt --no-deps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 38.3 MB/s eta 0:00:00m eta 0:00:010:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 MB 25.5 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 43.6 MB/s eta 0:00:00m eta 0:00:010:0101
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 67.1 MB/s eta 0:00:00m eta 0:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 11.0 MB/s eta 0:00:00
  Created wheel for overrides: filename=overrides-3.0.0-py3-none-any.whl size=5668 sha256=babf6ebfab3850bcfa200a293fefb3315789c541267531c4956be2a6ba8e356a
  Stored in directory: /root/.cache/pip/wheels/4a/62/c2/1a021999d160fce8988607bcd416f1da7a494b476ef3ce59a7
Successfully built overrides
  Attempting uninstall: opencv_python
    Found existing installation: opencv-python 4.7.0.68
    Uninstalling opencv-python-4.7.0.68:
      Successfully uninstalled opencv-python-4.7.0.68


In [6]:
#ImportError: libSM.so.6: cannot open shared object file: No such file or directory
!pip install opencv-python
!apt update && apt install -y libsm6 libxext6
!apt-get install -y libxrender-dev

Hit:1 http://repo.radeon.com/rocm/apt/3.5.1 xenial InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease                       
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease             
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
2 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsm6 is already the newest version (2:1.2.3-1).
libxext6 is already the newest version (2:1.3.4-0ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libxrender-dev is already the newest version (1:0.9.10-1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


In [7]:
import torch
torch.__version__

!python -m pip install torchtext==0.6 spacy thinc catalogue confection pydantic srsly wasabi blis cymem preshed murmurhash langcodes typer click boto3 botocore jmespath pytz  --no-deps
!python -m pip install allennlp==1.0.0 transformers==3.3.0 sentencepiece sacremoses joblib jsonpickle h5py nltk scipy threadpoolctl protobuf==3.20.0 tensorboardX --no-deps

In [ ]:
#!/bin/bash
!python -m torch.distributed.launch --nnode=1 --node_rank=0 --nproc_per_node=1 \
           train.py -c config.json -d 0 --local_world_size 1
#!python3 train_.py #--resume /content/drive/MyDrive/Colab_Notebooks/NLP/KG_KE/PICK-pytorch/saved/models/PICK_Default/test_0118_234222/model_best.pth ##uncomment for resume training
#!python train_.py #--resume /content/drive/MyDrive/Colab_Notebooks/NLP/KG_KE/PICK-pytorch/saved/models/PICK_Default/test_0118_234222/model_best.pth ##uncomment for resume training

In [9]:
!bash dist_train.sh

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
[W Module.cpp:473] Warning: Disabling benchmark mode for MIOpen is NOT supported. Overriding value to True (function operator())
[W Module.cpp:473] Warning: Disabling benchmark mode for MIOpen is NOT supported. Overriding value to True (function operator())
[W Module.cpp:473] Warning: Disabling benchmark mode for MIOpen is NOT supported. Overriding value to True (function operator())
[W Module.cpp:473] Warning: Disabling benchmark mode for MIOpen is NOT supported. Overriding value to True (function operator())
[W Module.cpp:473] Warning: Disabling benchmark mode for MIOpen is NOT supported. Overriding value to True (function operator())
[2023-02-07 21:56:31,676 - train - INFO] - Distributed GPU tr